# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [35]:
#Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('homepage_actions.csv')
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [36]:
df['action'].value_counts()

action
view     6328
click    1860
Name: count, dtype: int64

In [45]:
click_ids = set(df[df.action=='click']['id'].unique())
view_ids = set(df[df.action=='view']['id'].unique())
print("# of viewers: {} \t# of clickers: {}".format(len(view_ids), len(click_ids)))
print("# of viewers who didn't click: {}".format(len(view_ids - click_ids)))
print("# of clickers who didn't view: {}".format(len(click_ids - view_ids)))

# of viewers: 6328 	# of clickers: 1860
# of viewers who didn't click: 4468
# of clickers who didn't view: 0


In [38]:
"""
Everyone who clicked, also viewed the homepage
"""

'\nEveryone who clicked, also viewed the homepage\n'

In [53]:
df.group.value_counts()

group
control       4264
experiment    3924
Name: count, dtype: int64

In [55]:
experiment_ids = set(df[df.group=='experiment']['id'].unique())
control_ids = set(df[df.group=='control']['id'].unique())
print("Overlap of experiment and control groups: {}".format(len(control_ids & experiment_ids)))

Overlap of experiment and control groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [56]:
#Your code here
df['count'] = 1
df

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1
...,...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view,1
8184,2017-01-18 09:42:12.844575,755912,experiment,view,1
8185,2017-01-18 10:01:09.026482,458115,experiment,view,1
8186,2017-01-18 10:08:51.588469,505451,control,view,1


In [59]:
control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

In [84]:
df['count'].unique()

array([1])

In [85]:
experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)

In [88]:
df['group'].value_counts()

group
control       4264
experiment    3924
Name: count, dtype: int64

In [99]:
print("Sample size of control: {}".format(len(control)))
print("Sample size of experiment: {}".format(len(experiment)))
print(" ")
print("Total clicks for control: {}".format(control.click.sum()))
print("Toal clicks for experiment: {}".format(experiment.click.sum()))
print(" ")
print("Average click rate for control: {}".format(control.click.mean()))
print("Average click rate for experiment: {}".format(experiment.click.mean()))

Sample size of control: 3332
Sample size of experiment: 2996
 
Total clicks for control: 932.0
Toal clicks for experiment: 928.0
 
Average click rate for control: 0.2797118847539016
Average click rate for experiment: 0.3097463284379172


In [109]:
import numpy as np
import scipy.stats as stats

def welch_t(a, b):
    """Calculate Welch's t statistic for two samples."""

    numerator = a.mean() - b.mean()

    denominator = np.sqrt(a.var(ddof=1)/a.size + b.var(ddof=1)/b.size)

    return np.abs(numerator/denominator)

In [110]:
def welch_df(a, b):
    """ Calculate the effective degrees of freedom for two samples. 
    This function returns the degrees of freedom """

    s1 = a.var(ddof=1)
    s2 = b.var(ddof=2)
    n1 = a.size
    n2 = b.size

    numerator = (s1/n1 + s2/n2)**2
    denominator = (s1/n1)**2/(n1-1) + (s2/n2)**2/(n2-1)

    return numerator/denominator

In [111]:
def p_value_welch_ttest(a, b, two_sided=False):
    """Calculates the p-value for Welch's t-test give two samples.
    By default, the returned p-value is for a one-sided t-test.
    Set the two sided parameter to True if yo wish to perform a two-sided t-test instead."""

    t = welch_t(a, b)
    df = welch_df(a, b)

    p = 1 - stats.t.cdf(np.abs(t), df)

    if two_sided:
        return 2*p
    else:
        return p

In [112]:
p_value_welch_ttest(control.click, experiment.click)

0.004466403292638921

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [115]:
#Your code here
control_rate = control.click.mean()
expected_experiment_clicks_under_null = control_rate * len(experiment)
expected_experiment_clicks_under_null

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [117]:
#Your code here
n = len(experiment)
p = control_rate
var = n * p * (1-p)
std = np.sqrt(var)
std

24.568547907005815

In [119]:
actual_experiment_clicks = experiment.click.sum()
z_score = (actual_experiment_clicks - expected_experiment_clicks_under_null)/std
z_score

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [120]:
#Your code here
import scipy.stats as stats
p_val = stats.norm.sf(z_score) #OR WE CAN USE 1-stats.norm.cdf(z_score)
p_val

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**
>
> Yes, while the p-value is slightly lower, both would lead to a confident rejection of null hypothesis. The experimental page appears to be a more effective design. 

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.